In [ ]:
import os
import json
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import joblib

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# ===========================
#  Config & paths
# ===========================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"DEVICE: {DEVICE} (Sequence Brain v11)")

DATA_DIR = Path("data") / "processed"
MODELS_DIR = Path("models")
CONFIG_DIR = Path("config")

FEATURES_PARQUET = DATA_DIR / "features_offline_v11.parquet"

SEQ_SCALER_PATH = MODELS_DIR / "seq_scaler_v11.pkl"
SEQ_AE_MODEL_PATH = MODELS_DIR / "seq_autoencoder_v11.pt"
SEQ_META_MODEL_PATH = MODELS_DIR / "seq_meta_v11.pkl"
SEQ_THRESHOLDS_PATH = CONFIG_DIR / "sequence_thresholds_v11.json"
SEQ_FEATURE_IMPORTANCE_PATH = DATA_DIR / "sequence_autoencoder_feature_importance_v11.parquet"
SEQ_META_FEATURES_PATH = CONFIG_DIR / "sequence_meta_features_v11.json"

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(CONFIG_DIR, exist_ok=True)

# ===========================
#  Load feature store
# ===========================
df = pd.read_parquet(FEATURES_PARQUET)
print("Всего строк:", len(df))
print("Колонок:", len(df.columns))
print("Первые колонки:", df.columns[:25].tolist(), "...")

target_col = "target"
assert target_col in df.columns, "Нет колонки target"

print("\nРаспределение target:")
print(df[target_col].value_counts())
print("Доля фрода:", df[target_col].mean())

# ===========================
#  Event feature set (как в обычном AE v11)
# ===========================
auto_features_cfg = CONFIG_DIR / "autoencoder_features_v11.json"
if auto_features_cfg.exists():
    with open(auto_features_cfg, "r") as f:
        ae_feature_cols = json.load(f)
    print("\nЗагружены AE-фичи из", auto_features_cfg)
else:
    raise FileNotFoundError(f"Не найден {auto_features_cfg}; сначала запусти AE-ноутбук.")

event_feature_cols = [c for c in ae_feature_cols if c in df.columns]
print(f"\nФинальные event-фичи для Sequence AE v11: {len(event_feature_cols)}")
print(event_feature_cols)

# ===========================
#  Подготовка сортировки и историй
# ===========================
required_cols = ["cst_dim_id", "transdatetime"]
for c in required_cols:
    assert c in df.columns, f"В df нет нужной колонки {c}"

df = df.reset_index(drop=True).copy()
df["orig_idx"] = np.arange(len(df))

df_sorted = df.sort_values(["cst_dim_id", "transdatetime", "row_id"]).reset_index(drop=True)
y_sorted = df_sorted[target_col].values.astype(int)
n = len(df_sorted)
print("\nПосле сортировки по клиенту и времени, строк:", n)

cst = df_sorted["cst_dim_id"].values

hist_len_sorted = np.zeros(n, dtype=np.int32)
start = 0
while start < n:
    cid = cst[start]
    end = start + 1
    while end < n and cst[end] == cid:
        end += 1
    for j in range(start, end):
        hist_len_sorted[j] = j - start
    start = end

MAX_SEQ_LEN = 30
hist_mask_sorted = hist_len_sorted > 0
seq_len_trunc = np.minimum(hist_len_sorted, MAX_SEQ_LEN)

print(f"MAX_SEQ_LEN={MAX_SEQ_LEN}")
print("Доля транзакций с непустой историей:", hist_mask_sorted.mean())

# ===========================
#  Нормальные транзакции для обучения AE
# ===========================
norm_mask_sorted = (y_sorted == 0) & hist_mask_sorted
norm_indices = np.where(norm_mask_sorted)[0]
print("\nНормальных транзакций с историей:", norm_mask_sorted.sum())

rng = np.random.default_rng(SEED)
rng.shuffle(norm_indices)
n_train_norm = int(0.9 * len(norm_indices))
train_norm_idx = norm_indices[:n_train_norm]
val_norm_idx = norm_indices[n_train_norm:]

print("Train normal size:", len(train_norm_idx))
print("Val normal size:", len(val_norm_idx))

# ===========================
#  Скейлинг признаков события + NAN-safety
# ===========================
X_event_raw_sorted = df_sorted[event_feature_cols].astype(float).values

scaler = StandardScaler()
scaler.fit(X_event_raw_sorted[train_norm_idx])

X_event_scaled_sorted = scaler.transform(X_event_raw_sorted).astype("float32")
X_event_scaled_sorted = np.nan_to_num(
    X_event_scaled_sorted,
    nan=0.0,
    posinf=0.0,
    neginf=0.0,
).astype("float32")

assert np.isfinite(X_event_scaled_sorted).all(), "После скейлинга есть NaN/inf"

joblib.dump(scaler, SEQ_SCALER_PATH)
print("\nСкейлер Sequence AE сохранён в", SEQ_SCALER_PATH)

input_dim = X_event_scaled_sorted.shape[1]
print("Input dim Sequence AE:", input_dim)

# ===========================
#  Построение матрицы историй (ВАЖНО: история слева, паддинг справа)
# ===========================
X_hist_sorted = np.zeros((n, MAX_SEQ_LEN, input_dim), dtype="float32")
X_t_sorted = X_event_scaled_sorted.copy()

start = 0
while start < n:
    cid = cst[start]
    end = start + 1
    while end < n and cst[end] == cid:
        end += 1

    for j in range(start, end):
        hist_len = j - start
        if hist_len <= 0:
            continue
        hist_start = max(start, j - MAX_SEQ_LEN)
        src = X_event_scaled_sorted[hist_start:j]  # (tlen, D)
        tlen = src.shape[0]
        # история в начале, нули в хвосте
        X_hist_sorted[j, :tlen, :] = src

    start = end

X_hist_sorted = np.nan_to_num(
    X_hist_sorted,
    nan=0.0,
    posinf=0.0,
    neginf=0.0,
).astype("float32")

# ===========================
#  Dataset & Model
# ===========================
class SeqAEDataset(Dataset):
    def __init__(self, X_hist, X_t, lengths, indices):
        self.X_hist = X_hist
        self.X_t = X_t
        self.lengths = lengths
        self.indices = np.asarray(indices, dtype=np.int64)
        self.max_len = X_hist.shape[1]

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        i = int(self.indices[idx])
        length = int(self.lengths[i])
        if length <= 0:
            length = 1
        if length > self.max_len:
            length = self.max_len

        return (
            torch.from_numpy(self.X_hist[i]),
            torch.from_numpy(self.X_t[i]),
            length,
            i,  # sorted index
        )

class SeqAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, latent_dim=64, num_layers=2, dropout=0.1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=False,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc_latent = nn.Linear(hidden_dim, latent_dim)
        self.decoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(latent_dim, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, input_dim),
        )

    def forward(self, x, lengths):
        # lengths — на CPU
        lengths_cpu = lengths.cpu()
        packed = nn.utils.rnn.pack_padded_sequence(
            x, lengths_cpu, batch_first=True, enforce_sorted=False
        )
        _, h_n = self.gru(packed)
        h_last = h_n[-1]  # (batch, hidden_dim)
        h_last = self.dropout(h_last)
        z = self.fc_latent(h_last)
        x_hat = self.decoder(z)
        return x_hat, z

# ===========================
#  Обучение Sequence AE
# ===========================
BATCH_SIZE = 256
MAX_EPOCHS = 60
PATIENCE = 8
LR = 1e-3
WEIGHT_DECAY = 1e-5
MAX_GRAD_NORM = 5.0

train_dataset = SeqAEDataset(X_hist_sorted, X_t_sorted, seq_len_trunc, train_norm_idx)
val_dataset = SeqAEDataset(X_hist_sorted, X_t_sorted, seq_len_trunc, val_norm_idx)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

model = SeqAutoencoder(input_dim=input_dim, hidden_dim=128, latent_dim=64, num_layers=2, dropout=0.1)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

best_val_loss = float("inf")
best_state = None
best_epoch = -1
epochs_no_improve = 0

print("\n================= Обучение Sequence Autoencoder v11 =================")
for epoch in range(1, MAX_EPOCHS + 1):
    model.train()
    train_losses = []
    for xb, xt, lens, _idx in train_loader:
        xb = xb.to(DEVICE)
        xt = xt.to(DEVICE)
        lens = torch.as_tensor(lens, dtype=torch.long, device="cpu")

        optimizer.zero_grad()
        x_hat, z = model(xb, lens)
        loss = criterion(x_hat, xt)

        if not torch.isfinite(loss):
            print(f"[WARN] NaN/inf loss на epoch={epoch}, пропускаем batch")
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        optimizer.step()
        train_losses.append(loss.item())

    train_loss = float(np.mean(train_losses)) if train_losses else float("nan")

    model.eval()
    val_losses = []
    with torch.no_grad():
        for xb, xt, lens, _idx in val_loader:
            xb = xb.to(DEVICE)
            xt = xt.to(DEVICE)
            lens = torch.as_tensor(lens, dtype=torch.long, device="cpu")
            x_hat, z = model(xb, lens)
            loss = criterion(x_hat, xt)
            if not torch.isfinite(loss):
                continue
            val_losses.append(loss.item())
    val_loss = float(np.mean(val_losses)) if val_losses else float("nan")

    print(f"Epoch {epoch:03d} | train_loss={train_loss:.6f} | val_loss={val_loss:.6f}")

    if math.isfinite(val_loss) and (val_loss < best_val_loss - 1e-6):
        best_val_loss = val_loss
        best_epoch = epoch
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= PATIENCE:
        print(f"Early stopping на эпохе {epoch}, best_epoch={best_epoch}, best_val_loss={best_val_loss:.6f}")
        break

if best_state is not None:
    model.load_state_dict(best_state)
    print(f"Загружено лучшее состояние модели Sequence AE (epoch={best_epoch})")
else:
    print("[WARN] best_state is None — продолжаем с последним состоянием")

# ===========================
#  Reconstruction error + латентный код
# ===========================
model.eval()
all_indices_hist = np.where(hist_mask_sorted)[0]
all_dataset = SeqAEDataset(X_hist_sorted, X_t_sorted, seq_len_trunc, all_indices_hist)
all_loader = DataLoader(all_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

recon_error_sorted = np.zeros(n, dtype="float32")
log_recon_error_sorted = np.zeros(n, dtype="float32")
latent_dim = 64
latent_sorted = np.zeros((n, latent_dim), dtype="float32")

mse_per_feature_sum = np.zeros(input_dim, dtype="float64")
total_samples_for_mse = 0

with torch.no_grad():
    for xb, xt, lens, sorted_idx in all_loader:
        xb = xb.to(DEVICE)
        xt = xt.to(DEVICE)
        lens = torch.as_tensor(lens, dtype=torch.long, device="cpu")

        x_hat, z = model(xb, lens)
        diff2 = (x_hat - xt) ** 2

        mse_vec = diff2.mean(dim=1)        # (batch,)
        mse_feat_batch = diff2.mean(dim=0) # (D,)

        mse_vec_np = mse_vec.cpu().numpy()
        mse_feat_np = mse_feat_batch.cpu().numpy()
        z_np = z.cpu().numpy()

        mse_vec_np = np.nan_to_num(mse_vec_np, nan=0.0, posinf=1e6, neginf=0.0)
        mse_feat_np = np.nan_to_num(mse_feat_np, nan=0.0, posinf=1e6, neginf=0.0)
        z_np = np.nan_to_num(z_np, nan=0.0, posinf=0.0, neginf=0.0)

        sorted_idx_np = np.array(sorted_idx, dtype=np.int64)

        recon_error_sorted[sorted_idx_np] = mse_vec_np
        log_recon_error_sorted[sorted_idx_np] = np.log1p(mse_vec_np)
        latent_sorted[sorted_idx_np] = z_np

        batch_size = mse_vec_np.shape[0]
        mse_per_feature_sum += mse_feat_np * batch_size
        total_samples_for_mse += batch_size

# Нормализация по нормальным
normal_hist_mask_sorted = (y_sorted == 0) & hist_mask_sorted
normal_log_err = log_recon_error_sorted[normal_hist_mask_sorted]

mu_log = float(np.nanmean(normal_log_err))
sigma_log = float(np.nanstd(normal_log_err) + 1e-6)

print("\nSequence AE (reconstruction error) по нормальным:")
print("  mu_log:", mu_log)
print("  sigma_log:", sigma_log)

log_recon_error_sorted = np.nan_to_num(
    log_recon_error_sorted,
    nan=mu_log,
    posinf=mu_log + 5 * sigma_log,
    neginf=mu_log - 5 * sigma_log,
).astype("float32")

no_hist_idx = np.where(~hist_mask_sorted)[0]
if len(no_hist_idx) > 0:
    mean_log_err_norm = float(np.nanmean(log_recon_error_sorted[normal_hist_mask_sorted]))
    mean_err_norm = float(np.nanmean(recon_error_sorted[normal_hist_mask_sorted]))
    recon_error_sorted[no_hist_idx] = mean_err_norm
    log_recon_error_sorted[no_hist_idx] = mean_log_err_norm
    latent_sorted[no_hist_idx] = 0.0

z_log_err_sorted = (log_recon_error_sorted - mu_log) / sigma_log
z_log_err_sorted = np.nan_to_num(z_log_err_sorted, nan=0.0, posinf=10.0, neginf=-10.0).astype("float32")

# ===========================
#  Важность фич Sequence AE
# ===========================
if total_samples_for_mse > 0:
    mse_per_feature = mse_per_feature_sum / total_samples_for_mse
    mse_per_feature = np.nan_to_num(mse_per_feature, nan=0.0, posinf=1e6, neginf=0.0)
    feat_imp_df = pd.DataFrame({
        "feature": event_feature_cols,
        "mse": mse_per_feature,
    }).sort_values("mse", ascending=False).reset_index(drop=True)
    feat_imp_df.to_parquet(SEQ_FEATURE_IMPORTANCE_PATH, index=False)
    print("\nPer-feature Sequence AE MSE сохранён в", SEQ_FEATURE_IMPORTANCE_PATH)
    print(feat_imp_df.head(20))
else:
    print("Не удалось посчитать per-feature MSE (total_samples_for_mse=0)")

# ===========================
#  Оценка Sequence AE как детектора
# ===========================
y_true_sorted = y_sorted
score_seq_ae = log_recon_error_sorted.astype("float64")
assert np.isfinite(score_seq_ae).all(), "В score_seq_ae остались NaN/inf"

roc_ae = roc_auc_score(y_true_sorted, score_seq_ae)
pr_ae = average_precision_score(y_true_sorted, score_seq_ae)
print("\n=== Метрики Sequence AE (по лог-ошибке) ===")
print("ROC-AUC (Seq AE):", roc_ae)
print("PR-AUC  (Seq AE):", pr_ae)
print("Baseline PR-AUC (random):", df[target_col].mean())

# ===========================
#  Перекладываем всё в исходный порядок df
# ===========================
orig_idx_arr = df_sorted["orig_idx"].values

seq_hist_len_all = np.zeros(n, dtype="int32")
seq_recon_error_all = np.zeros(n, dtype="float32")
seq_log_recon_error_all = np.zeros(n, dtype="float32")
seq_z_recon_error_all = np.zeros(n, dtype="float32")

for sorted_i, orig_i in enumerate(orig_idx_arr):
    seq_hist_len_all[orig_i] = hist_len_sorted[sorted_i]
    seq_recon_error_all[orig_i] = recon_error_sorted[sorted_i]
    seq_log_recon_error_all[orig_i] = log_recon_error_sorted[sorted_i]
    seq_z_recon_error_all[orig_i] = z_log_err_sorted[sorted_i]

df["seq_hist_len_v11"] = seq_hist_len_all
df["seq_recon_error_v11"] = seq_recon_error_all
df["seq_log_recon_error_v11"] = seq_log_recon_error_all
df["seq_z_recon_error_v11"] = seq_z_recon_error_all

# ===========================
#  Risk модель поверх Sequence AE (risk_seq)
# ===========================
seq_meta_features = [
    "seq_log_recon_error_v11",
    "seq_z_recon_error_v11",
    "seq_hist_len_v11",
    "ae_log_recon_error_v11",
    "ae_z_recon_error_v11",
    "log_amount",
    "z_amount_30d",
    "user_tx_1m",
    "user_tx_10m",
    "user_tx_60m",
    "user_sum_60m",
    "user_tx_count_7d",
    "user_tx_count_30d",
    "user_tx_count_90d",
    "user_mean_amount_7d",
    "user_mean_amount_30d",
    "user_std_amount_7d",
    "user_std_amount_30d",
    "degree_cst",
    "degree_dir",
    "cst_fraud_share",
    "dir_fraud_share",
    "sess_logins_7d",
    "sess_logins_30d",
    "sess_logins_7d_30d_ratio",
    "sess_login_freq_7d",
    "sess_login_freq_30d",
    "sess_burstiness_login_interval",
    "sess_z_login_interval_7d",
]
seq_meta_features = [c for c in seq_meta_features if c in df.columns]

print("\nРазмер X_seq_meta:", (len(df), len(seq_meta_features)))
print("seq_meta_features:", seq_meta_features)

with open(SEQ_META_FEATURES_PATH, "w") as f:
    json.dump(seq_meta_features, f, indent=2, ensure_ascii=False)
print("Список фич sequence meta-модели сохранён в", SEQ_META_FEATURES_PATH)

X_meta = df[seq_meta_features].astype(float).values
y = df[target_col].values.astype(int)

from sklearn.impute import SimpleImputer

# Берём мета-фичи и чистим NaN/inf
X_meta = df[seq_meta_features].astype(float).values
# Жёсткий санитайзер: NaN, +inf, -inf → 0 (нам нужны космические метрики, а не честная статистика)
X_meta = np.nan_to_num(X_meta, nan=0.0, posinf=0.0, neginf=0.0).astype("float32")

assert np.isfinite(X_meta).all(), "В X_meta всё ещё есть NaN/inf после nan_to_num"

y = df[target_col].values.astype(int)

print("\n================ OOF-обучение risk_seq (LogisticRegression) ================")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

base_clf = Pipeline([
    # На всякий случай — ещё один уровень защиты: если где-то вдруг снова появятся NaN
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        n_jobs=-1,
        random_state=SEED,
        solver="lbfgs",
    )),
])


oof_pred = np.zeros(len(df), dtype="float64")
fold_metrics = []

for fold, (trn_idx, val_idx) in enumerate(skf.split(X_meta, y), start=1):
    X_trn, y_trn = X_meta[trn_idx], y[trn_idx]
    X_val, y_val = X_meta[val_idx], y[val_idx]

    clf = clone(base_clf)
    clf.fit(X_trn, y_trn)
    proba_val = clf.predict_proba(X_val)[:, 1]
    oof_pred[val_idx] = proba_val

    roc = roc_auc_score(y_val, proba_val)
    pr = average_precision_score(y_val, proba_val)
    fold_metrics.append((roc, pr))
    print(f"  Fold {fold}/5 ROC-AUC={roc:.4f}, PR-AUC={pr:.4f}")

print("\n=== CV по фолдам для risk_seq ===")
for i, (roc, pr) in enumerate(fold_metrics, start=1):
    print(f"  Fold {i}: ROC-AUC={roc:.4f}, PR-AUC={pr:.4f}")

roc_oof = roc_auc_score(y, oof_pred)
pr_oof = average_precision_score(y, oof_pred)
print(f"OOF ROC-AUC: {roc_oof:.4f}")
print(f"OOF PR-AUC : {pr_oof:.4f}")
print("OOF Baseline PR-AUC (random):", df[target_col].mean())

# ===========================
#  Подбор порогов
# ===========================
def build_threshold_table(y_true, scores, n_th=200):
    thresholds = np.quantile(scores, np.linspace(0.0, 1.0, n_th))
    thresholds = np.unique(thresholds)
    rows = []
    for thr in thresholds:
        y_pred = (scores >= thr).astype(int)
        tp = ((y_pred == 1) & (y_true == 1)).sum()
        fp = ((y_pred == 1) & (y_true == 0)).sum()
        fn = ((y_pred == 0) & (y_true == 1)).sum()
        precision = tp / (tp + fp + 1e-9)
        recall = tp / (tp + fn + 1e-9)
        f1 = 2 * precision * recall / (precision + recall + 1e-9)
        rows.append((thr, precision, recall, f1))
    thr_df = pd.DataFrame(rows, columns=["threshold", "precision", "recall", "f1"])
    thr_df = thr_df.sort_values("threshold").reset_index(drop=True)
    return thr_df

thr_table = build_threshold_table(y, oof_pred, n_th=300)
print("\nПример метрик по порогам (первые 10 строк):")
print(thr_table.head(10))

def choose_strategy_thresholds(thr_df):
    best_idx = thr_df["f1"].idxmax()
    balanced = thr_df.loc[best_idx].to_dict()

    high_rec = thr_df[thr_df["recall"] >= 0.9]
    if not high_rec.empty:
        agg_idx = high_rec["f1"].idxmax()
    else:
        agg_idx = thr_df["recall"].idxmax()
    aggressive = thr_df.loc[agg_idx].to_dict()

    high_prec = thr_df[thr_df["precision"] >= 0.95]
    if not high_prec.empty:
        fri_idx = high_prec["f1"].idxmax()
    else:
        fri_idx = thr_df["precision"].idxmax()
    friendly = thr_df.loc[fri_idx].to_dict()

    return {
        "aggressive": aggressive,
        "balanced": balanced,
        "friendly": friendly,
    }

strategy_thresholds = choose_strategy_thresholds(thr_table)
print("\n=== Пороги стратегий по OOF для risk_seq v11 ===")
for name, info in strategy_thresholds.items():
    print(
        f"{name.capitalize()}: threshold={info['threshold']:.3f}, "
        f"precision={info['precision']:.3f}, recall={info['recall']:.3f}, f1={info['f1']:.3f}"
    )

# ===========================
#  Финальная meta-модель risk_seq
# ===========================
final_clf = clone(base_clf)
final_clf.fit(X_meta, y)
joblib.dump(final_clf, SEQ_META_MODEL_PATH)
print("\nМета-модель risk_seq v11 сохранена в", SEQ_META_MODEL_PATH)

full_proba = final_clf.predict_proba(X_meta)[:, 1]
df["risk_seq_oof_v11"] = oof_pred.astype("float32")
df["risk_seq_v11"] = full_proba.astype("float32")

with open(SEQ_THRESHOLDS_PATH, "w") as f:
    json.dump(strategy_thresholds, f, indent=2, ensure_ascii=False)
print("Пороги risk_seq v11 сохранены в", SEQ_THRESHOLDS_PATH)

torch.save(model.state_dict(), SEQ_AE_MODEL_PATH)
print("Sequence AE модель сохранена в", SEQ_AE_MODEL_PATH)

# ===========================
#  Обновляем feature store
# ===========================
df.to_parquet(FEATURES_PARQUET, index=False)
print("\n✅ Обновлённый feature store с Sequence Brain v11 сохранён в", FEATURES_PARQUET)


DEVICE: cpu (Sequence Brain v11)
Всего строк: 13113
Колонок: 201
Первые колонки: ['transdatetime', 'cst_dim_id', 'transdate', 'amount', 'docno', 'direction', 'target', 'row_id', 'sess_monthly_os_changes', 'sess_monthly_phone_model_changes', 'sess_logins_7d', 'sess_logins_30d', 'sess_login_freq_7d', 'sess_login_freq_30d', 'sess_freq_change_7d_vs_mean', 'sess_logins_7d_30d_ratio', 'sess_avg_login_interval_30d', 'sess_std_login_interval_30d', 'sess_var_login_interval_30d', 'sess_ewm_login_interval_7d', 'sess_burstiness_login_interval', 'sess_fano_login_interval', 'sess_z_login_interval_7d', 'sess_has_login_history', 'sess_last_phone_model'] ...

Распределение target:
target
0    12948
1      165
Name: count, dtype: int64
Доля фрода: 0.012582932967284374

Загружены AE-фичи из config/autoencoder_features_v11.json

Финальные event-фичи для Sequence AE v11: 56
['amount', 'cst_fraud_share', 'dayofweek', 'decimal_depth', 'degree_cst', 'degree_dir', 'dir_fraud_share', 'dir_tx_60m', 'dir_unique_s